In [117]:
! lscpu

Arquitectura:                            x86_64
  modo(s) de operación de las CPUs:      32-bit, 64-bit
  Address sizes:                         39 bits physical, 48 bits virtual
  Orden de los bytes:                    Little Endian
CPU(s):                                  16
  Lista de la(s) CPU(s) en línea:        0-15
ID de fabricante:                        GenuineIntel
  Nombre del modelo:                     Intel(R) Core(TM) i7-10700 CPU @ 2.90GH
                                         z
    Familia de CPU:                      6
    Modelo:                              165
    Hilo(s) de procesamiento por núcleo: 2
    Núcleo(s) por «socket»:              8
    «Socket(s)»                          1
    Revisión:                            5
    CPU MHz máx.:                        4800,0000
    CPU MHz mín.:                        800,0000
    BogoMIPS:                            5799.77
    Indicadores:                         fpu vme de pse tsc msr pae mce cx8 apic
       

In [119]:
%%file suma_cubos.c
#include <math.h>
double suma_cubos(int * vector, int cantidad_elementos){
    double suma=0;
    for(int i=0; i<=cantidad_elementos-1;i++){
        suma=suma+ pow((double)vector[i],3.0);
    }
    return suma;
}

Overwriting suma_cubos.c


In [120]:
! gcc -Os -c suma_cubos.c -lm -o suma_cubos.o

In [121]:
! objdump -M intel -j .text -D suma_cubos.o


suma_cubos.o:     formato del fichero elf64-x86-64


Desensamblado de la sección .text:

0000000000000000 <suma_cubos>:
   0:	f3 0f 1e fa          	endbr64 
   4:	41 54                	push   r12
   6:	0f 57 d2             	xorps  xmm2,xmm2
   9:	49 89 fc             	mov    r12,rdi
   c:	55                   	push   rbp
   d:	89 f5                	mov    ebp,esi
   f:	53                   	push   rbx
  10:	31 db                	xor    ebx,ebx
  12:	48 83 ec 10          	sub    rsp,0x10
  16:	39 dd                	cmp    ebp,ebx
  18:	7e 2c                	jle    46 <suma_cubos+0x46>
  1a:	f2 41 0f 2a 04 9c    	cvtsi2sd xmm0,DWORD PTR [r12+rbx*4]
  20:	48 8b 05 00 00 00 00 	mov    rax,QWORD PTR [rip+0x0]        # 27 <suma_cubos+0x27>
  27:	f2 0f 11 54 24 08    	movsd  QWORD PTR [rsp+0x8],xmm2
  2d:	48 ff c3             	inc    rbx
  30:	66 48 0f 6e c8       	movq   xmm1,rax
  35:	e8 00 00 00 00       	call   3a <suma_cubos+0x3a>
  3a:	f2 0f 10 54 24 08    	movsd  xmm2,QWORD PTR [rsp+0

In [230]:
%%file main.c
#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#include <time.h>
#include <x86intrin.h>
//Usamos la variable tipo int porque la funcion rand generara numeros aleatorios enteros entre 0 y 9
int * Crear_arr( int N){
    int *array = (int *)malloc(N*sizeof(int));
    for(int i=0; i<=N-1; i++){
        array[i]=rand()%9;
        //printf("%d ", array[i]);
    return array;
}
}
//Usamos double porque tendremos sumas grandes
double suma_cubos(int *vector, int cantidad_elementos){
    double suma=0;
    for(int i=0; i<=cantidad_elementos-1;i++){
        suma=suma+ pow((double)vector[i],3.0);
    }
    return suma;
}
//se puede deducir que la cantidad de instrucciones es de Int= 7+9*n
//Usaremos int porque solo trabajamos con enteros
int num_instrucciones(int N){
    return 7+N*9;
}
//Usamos long int en la entrada porque la diferencia entre ciclos suelen ser numeros muy grandes pero enteros
double Calcular_CPI(long int cantidad_ciclos, int cantidad_instrucciones){
    return (double)cantidad_ciclos/(double)cantidad_instrucciones;
}
//Usamos double porque los mips son divisiones
double Calcular_MIPS(double frecuencia, double CPI){
    return (double)frecuencia/(CPI*pow(10,6));
}
//Usamos void porque la funcion imprime varios valores
void Test( int elementos_arreglo){
    int *array;
    srand(time(NULL));
    double frecuencia =((4800+800)/2.0)*pow(10,6);
    int numero_instrucciones=num_instrucciones(elementos_arreglo);
    long int tic, toc, ciclo;
    double CPI, MIPS, min_MIPS=999.0, max_MIPS=0.0, min_CPI=999.0, max_CPI=0.0, prom_MIPS=0.0, prom_CPI=0.0;
    //Hacemos que la iterativa se repita 100 000 veces para comparar los CPI y MIPS en cada ejecucion al generar sumas de cubos de numeros randoms
    for(int i=1; i<=100000; i++){
        array= Crear_arr(elementos_arreglo);
        tic= __rdtsc();
        suma_cubos(array, elementos_arreglo);
        toc= __rdtsc();
        ciclo= toc-tic;
        CPI=Calcular_CPI(ciclo, numero_instrucciones);
        MIPS=Calcular_MIPS(frecuencia, CPI);
        //printf("MIPS ES %lf\n", MIPS);
        //printf("frecuencia es %lf\n", frecuencia);
        //printf("cpi es %lf\n", CPI);
        prom_CPI=prom_CPI+CPI;
        prom_MIPS=prom_MIPS+MIPS;
        if(CPI<=min_CPI){
            min_CPI=CPI;
        }
        if(CPI>=max_CPI){
            max_CPI=CPI;
        }
        if(MIPS>=max_MIPS){
            max_MIPS=MIPS;
        }
        if(MIPS<=min_MIPS){
            min_MIPS=MIPS;
        }
    }
    prom_CPI=prom_CPI/100000.0;
    prom_MIPS=prom_MIPS/100000.0;
    printf("El máximo valor del CPI es %lf\n", max_CPI);
    printf("EL minimo valor del CPI es %lf\n", min_CPI);
    printf("EL valor promedio del CPI es %lf\n", prom_CPI);
    printf("El máximo valor del MIPS es %lf\n", max_MIPS);
    printf("EL minimo valor del MIPS es %lf\n", min_MIPS);
    printf("EL valor promedio del MIPS es %lf\n", prom_MIPS);
}


int main(int argc, char const *argv[]){
    int N;
    if(argc==1){
        Test(16384);
    }
    if(argc==2){
        N=atoi(argv[1]);
        Test(N);
    }
    if(argc>2){
        printf("Está pasando más de la cantidad de los argumentos requeridos\n");
        printf("Debe ingresar un valor para la cantidad de elementos del arreglo\n");
        return 0;
    }
    return 0;
}


Overwriting main.c


In [231]:
! gcc main.c -lm -o main

In [232]:
! ./main 

El máximo valor del CPI es 4.750331
EL minimo valor del CPI es 1.886419
EL valor promedio del CPI es 1.983925
El máximo valor del MIPS es 1484.293813
EL minimo valor del MIPS es 589.432661
EL valor promedio del MIPS es 1415.539081


In [233]:
! ./main 1 1

Está pasando más de la cantidad de los argumentos requeridos
Debe ingresar un valor para la cantidad de elementos del arreglo


In [234]:
! ./main 1 1 1

Está pasando más de la cantidad de los argumentos requeridos
Debe ingresar un valor para la cantidad de elementos del arreglo


In [235]:
! ./main 32768

El máximo valor del CPI es 3.964404
EL minimo valor del CPI es 1.921860
EL valor promedio del CPI es 1.993717
El máximo valor del MIPS es 1456.922016
EL minimo valor del MIPS es 706.285271
EL valor promedio del MIPS es 1408.192986


In [236]:
! ./main 32768

El máximo valor del CPI es 7.744998
EL minimo valor del CPI es 1.920653
EL valor promedio del CPI es 2.006401
El máximo valor del MIPS es 1457.837677
EL minimo valor del MIPS es 361.523667
EL valor promedio del MIPS es 1401.035067


In [237]:
! ./main 65536

El máximo valor del CPI es 8.286345
EL minimo valor del CPI es 1.927671
EL valor promedio del CPI es 1.995022
El máximo valor del MIPS es 1452.530167
EL minimo valor del MIPS es 337.905324
EL valor promedio del MIPS es 1406.052634


In [239]:
! ./main 65536

El máximo valor del CPI es 6.019633
EL minimo valor del CPI es 1.891096
EL valor promedio del CPI es 1.991350
El máximo valor del MIPS es 1480.622936
EL minimo valor del MIPS es 465.144656
EL valor promedio del MIPS es 1408.491226
